Property finder relases all the data 

<h3>Scrapping all ad pages  </h3>

<h5>When using Headless option you have to add user agent 
 because Headless mode uses its own default User-Agent if it is not given as an argument.
 However some webpages may block Headless mode User-Agent to avoid unwanted traffic.
 It may result in Access denied error while trying to open a webpage. </h5>

In [ ]:
print(f'Total Ads Scrapped:{len(ads_pages)}')

# Open a new CSV file for writing
with open('Property_finder_ad_pages_final.csv', 'w') as f:
    # Create a CSV writer object
    writer = csv.writer(f)
    # Write each link to a new row in the file
    writer.writerows([[link] for link in ads_pages])

chrome.quit()


<h3>Set options<h3>

In [ ]:
import csv
from math import ceil
from random import sample
from time import sleep
from bs4 import BeautifulSoup
from requests import get
from selenium.common.exceptions import TimeoutException
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
import re
import pandas as pd 
import numpy as np 
from collections import deque
import joblib

# Setting user agent string to prevent bot detection
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'

def set_options(*args):
    # Creating Selenium options object with default settings
    options = Options()
    
    # Disabling images to speed up page loading
    prefs = {'profile.managed_default_content_settings.images': 2}
    options.add_experimental_option('prefs', prefs)
    
    # Adding any additional arguments to the options object
    for arg in args:
        options.add_argument(arg)
        
    # Setting page load strategy to 'eager' to wait for DOMContentLoaded event
    # This is because the items being scraped are loading quickly
    options.page_load_strategy = 'eager'

    # Returning the modified options object
    return options

Scrapping all properties 

In [2]:
# Get all Home pages links
def scrapp_all_properties():
    page_links = [f'https://www.propertyfinder.eg/en/buy/properties-for-sale.html?page={i}' for i in range(1, 4000)]

    chrome = Chrome(ChromeDriverManager().install(), options=set_options('--headless','--ignore-certificate-errors','--incognito',"--disable-gpu","--no-sandbox","--disable-extensions","--dns-prefetch-disable","start-maximized","enable-automation","--disable-dev-shm-usage","--disable-browser-side-navigation","--disable-infobars",f'user-agent={user_agent}'))
    counter=0
    ads_pages = []
    total_ads_scrapped = 0

    # Send a GET request to the webpage
    for i in page_links:
        try:
            chrome.get(i)
        except TimeoutException as e:
            print(f"Failed to load page: {i}. Error: {e}")
            continue
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(chrome.page_source, 'html.parser')
        # Find and print all links on the page
        links = soup.select('a.card__link')
        ads_pages.extend(links)
        print(f'links visited:{chrome.current_url.split("=")[1]}')
        #Counter to exit loop to test 
        # counter=counter+1
        # if counter==1:
        #     break
    print(f'Total Ads Scrapped:{len(ads_pages)}')
    #Convert it to set to remove duplicated links
    set(ads_pages)
    # joblib.dump(ads_pages, 'state2.pkl')
    # This line is creating two empty lists, 'href' and 'title', to which we will append values in the loop below.
    # This line iterates over each element 'x' in the list 'ads_pages'. 
    #!For each element x it extracts the 'href' attribute and appends it to the 'href' list, and also extracts the 'title' attribute and appends it to the 'title' list. 
    #!The results are stored as tuples inside the respective lists.


    # Create an empty deque
    href_deque = deque()

    # Use list comprehension to append items to deque
    ([(href_deque.appendleft(x.get('href'))) for x in ads_pages])
    joblib.dump(href_deque, 'href_deque_state.pkl')
    joblib.dump(href_deque, 'href_deque_state_full.pkl')

    chrome.quit()
  
scrapp_all_properties()

[WDM] - Downloading: 100%|██████████| 6.80M/6.80M [00:02<00:00, 3.38MB/s]
C:\Users\amigo\AppData\Local\Temp\ipykernel_44684\2265430753.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  chrome = Chrome(ChromeDriverManager().install(), options=set_options('--headless','--ignore-certificate-errors','--incognito',"--disable-gpu","--no-sandbox","--disable-extensions","--dns-prefetch-disable","start-maximized","enable-automation","--disable-dev-shm-usage","--disable-browser-side-navigation","--disable-infobars",f'user-agent={user_agent}'))


links visited:1
Total Ads Scrapped:25


<h4>Scrapp Ad Pages </h4>

In [9]:
# Create an empty list to store the amenities
# Import the necessary libraries and functions 
import joblib
from collections import deque
from selenium.webdriver import Chrome
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import re

amenities_list = [] # Initialize an empty list to hold scraped amenities.
chrome = Chrome(ChromeDriverManager().install(), options=set_options('--headless','--ignore-certificate-errors','--incognito',"--disable-gpu","--no-sandbox","--disable-extensions","--dns-prefetch-disable","start-maximized","enable-automation","--disable-dev-shm-usage","--disable-browser-side-navigation","--disable-infobars",f'user-agent={user_agent}')) # Create a new headless chrome instance.

try:    
    href_deque = joblib.load('href_deque_state.pkl') # Load the URL queue from file.
    ads_list_dic=joblib.load('ads_scrapped.pkl') # Deserialize ads data previously scraped.
except:
    #DONE not logic to scrapp all again 
    # scrapp_all_properties()
    href_deque = joblib.load('href_deque_state.pkl') # Load the URL queue from file.

finally:
    while href_deque:
        current_url = href_deque.popleft() # Get the next URL in the queue.
        ad={'url':current_url} # Create a dictionary object to hold scraped data for this URL.

        print(current_url) # Print current URL being scraped.
        scraped_urls = {x['url'] for x in ads_list_dic} # Make a set from URLs of previously scraped properties.
        
        if current_url not in scraped_urls: # Check if this property has already been scraped.
            print(len(ads_list_dic), current_url)
            while True:
                try:
                    chrome.get(current_url) # Navigate the scraper to the URL.
                    break
                except TimeoutException:
                    print('timeout detected... cooling off...')
                    sleep(2.0)
                    
            soup = BeautifulSoup(chrome.page_source, 'html.parser') # Create a new Beautiful Soup object from the page source.
            
            ad['title'] = soup.find('h1',class_ = 'property-page__title').text.strip() # Scrape the title of the property from the parsed HTML.
            ad['sub_title'] = soup.find('h2',class_ = 'property-page__sub-title').text.strip() # Scrape the subtitle of the property from the parsed HTML.
            li = soup.find_all('div',class_ = 'property-facts__value') # Find all divs with the "property-facts__value" class.
            ad['property_type']=soup.find_all('div',class_ = 'property-facts__value')[0].text.strip() # Scrape the type of the property from the HTML.
            property_size_temp=soup.find_all('div',class_ = 'property-facts__value')[1].text.strip() # Scrape the raw size value (e.g. "1234 sqft") from the HTML.
            ad['property_size']=re.sub(r'\D*(\d+)\D*',r'\1', li[1].text.strip()[15:]).strip() # Parse only the numeric part of the raw size value and assign the result to the 'property_size' key in the dictionary.
            ad['bedrooms']=soup.find_all('div',class_ = 'property-facts__value')[2].text.replace('\n','').replace('\t', '').strip() # Scrape the number of bedrooms from the HTML.
            ad['bathrooms']=soup.find_all('div',class_ = 'property-facts__value')[3].text.strip() # Scrape the number of bathrooms from the HTML.
            ad['agent_name']=soup.find('h4',class_ = 'property-agent__name').text.strip() # Scrape the name of the agent from the HTML.
            ad['agent_company']=soup.find('div',class_ = 'property-agent__position-broker-name').text.strip() # Scrape the name of the agent's company from the HTML.
            ad['agent_number'] = soup.find('a', class_='property-contact__primary-button').get('href')[4:] # Scrape the agent's phone number from the HTML.
            ad['description']=soup.find('div',class_ = 'property-description__text-trim').text.replace('\n','').strip() # Scrape the description from the HTML.
            ad['listed_date']=soup.find_all('div',class_ = 'property-page__legal-list-content')[1].text.strip() # Scrape the listing date from the HTML.
            ad['price']=soup.find('div',class_ = 'property-price__price').text.strip() # Scrape the price of the property from the HTML.
            ad['date']=soup.find_all('div',class_ = 'property-page__legal-list-content')[1].text.strip() # Scrape the date from the HTML.
            #! Ameneties 
            amenities_all_divs = soup.find_all('div', class_='property-amenities__list') # Find all divs with the "property-amenities__list" class.
            [amenities_list.append(x.text.strip()) for x in amenities_all_divs] # Add the text content of each "property-amenities__list" div (with leading/trailing whitespace removed) to the amenities_list.
            ad['amenities'] = ";".join(amenities_list) # Join all elements of the amenities_list together into a single string, separated by semicolons (;), and assign the result to the 'amenities' key in the dictionary.

            ads_list_dic.append(ad) # Append this dictionary object to the list of scraped properties.
            joblib.dump(ads_list_dic, 'ads_scrapped.pkl') # Serialize the updated list of scraped properties to file.
            joblib.dump(href_deque, 'href_deque_state.pkl') # Serialize the updated deque of URLs still to scrape to file.
            
chrome.quit() # Quit the headless chrome browser instance. 


C:\Users\amigo\AppData\Local\Temp\ipykernel_22904\358929997.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  chrome = Chrome(ChromeDriverManager().install(), options=set_options('--headless','--ignore-certificate-errors','--incognito',"--disable-gpu","--no-sandbox","--disable-extensions","--dns-prefetch-disable","start-maximized","enable-automation","--disable-dev-shm-usage","--disable-browser-side-navigation","--disable-infobars",f'user-agent={user_agent}'))


<h4>Save to CSV and check all scrapped</h4>

In [ ]:
print(f'Total Ads Scrapped:{len(ads_pages)}')

# Open a new CSV file for writing
with open('All_properties_property_finder_ad_pages.csv', 'wb') as f:
    # Create a CSV writer object
    writer = csv.writer(f)
    # Write each link to a new row in the file
    writer.writerows([str(link)] for link in ads_pages)

chrome.quit()

In [8]:
aa=joblib.load('ads_scrapped.pkl')
# convert the list of tuples to a DataFrame
df = pd.DataFrame(aa)

# print the DataFrame
df


,url,title,sub_title,property_type,property_size,bedrooms,bathrooms,agent_name,agent_company,agent_number,description,listed_date,amenities
0,https://www.propertyfinder.eg/en/plp/buy/duple...,Duplex For Sale Fully Finished in IRIS Swan La...,"Duplex for Sale in Swan Lake Residence, 5th Se...",Duplex,319,3,3,marina Samy,URE-Ultimate Real Estate,+201090601060,Swan Lake ResidenceIRISDuplex Ground + FirstFu...,2 months ago,Unfurnished;Security;Balcony;Shared Gym;Shared...
1,https://www.propertyfinder.eg/en/plp/buy/duple...,Duplex For Sale Fully Finished in IRIS Swan La...,"Duplex for Sale in Swan Lake Residence, 5th Se...",Duplex,319,3,3,marina Samy,URE-Ultimate Real Estate,+201090601060,Swan Lake ResidenceIRISDuplex Ground + FirstFu...,2 months ago,Unfurnished;Security;Balcony;Shared Gym;Shared...
2,https://www.propertyfinder.eg/en/plp/buy/duple...,Duplex For Sale Fully Finished in IRIS Swan La...,"Duplex for Sale in Swan Lake Residence, 5th Se...",Duplex,319,3,3,marina Samy,URE-Ultimate Real Estate,+201090601060,Swan Lake ResidenceIRISDuplex Ground + FirstFu...,2 months ago,Unfurnished;Security;Balcony;Shared Gym;Shared...
3,https://www.propertyfinder.eg/en/plp/buy/villa...,Villa for sale in VYE Sodic with 5 % down pay...,"Villa for Sale in Vye Sodic, New Zayed City",Villa,350,6,3,Smaher Hesham,TAI - The Address Investments,+201098716417,Fully finished villa for sale in New Vye SODIC...,6 months ago,Unfurnished;Security;Private Garden;Shared Gym...
4,https://www.propertyfinder.eg/en/plp/buy/duple...,duplex in Al-Andalus | Superlux | ready to move,"Duplex for Sale in Al Andalus El Gedida, Al An...",Duplex,264,3,3,Wall Street Group,Wall Street Group,+201002197002,Duplex for sale in Al Andalus ready to movesup...,8 days ago,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,https://www.propertyfinder.eg/en/plp/buy/townh...,Townhouse for sale In Mountain view October pa...,Townhouse for Sale in Mountain View October Pa...,Townhouse,210,3,4,Smaher Hesham,TAI - The Address Investments,+201098716417,Townhouse for sale In Mountain view October pa...,6 months ago,Unfurnished;Central A/C;Security;Children's Po...
123,https://www.propertyfinder.eg/en/plp/buy/apart...,OWN YOUR PREMIUM HURGHADA HOME,"Apartment for Sale in Al Nasr St., Hurghada",Apartment,81,studio,1,Eman Hosny,Aka Group Real estate,+201277283838,OWN YOUR PREMIUM HURGHADA HOME IN A STRATEGIC ...,1 month ago,Unfurnished;Central A/C;Security;Children's Po...
124,https://www.propertyfinder.eg/en/plp/buy/chale...,"Chalet with Garden , Hyde park North Coast","Chalet for Sale in Hyde Park North, Ras Al Hekma",Chalet,73,1+Maid,2,B2B Real Estate,B2B Real Estate Consultancy & Investment Advisory,19323,"The projects of the Hyde Park Company, specifi...",14 days ago,Unfurnished;Central A/C;Security;Children's Po...
125,https://www.propertyfinder.eg/en/plp/buy/apart...,400 sqm apartment for sale in Mohandessein,Apartment for Sale in Al Batal Ahmed Abd El Az...,Apartment,400,4+Maid,4,Fouad Darwish,Coldwell Banker Gate,+201223880669,Spacious 400 meter apartment for sale in the M...,8 days ago,Unfurnished;Central A/C;Security;Children's Po...
